<center>
<a href="http://www.udem.edu.co/"><img src="img/Escudo.png"></a>
<h1>Reconocimiento de Patrones I</h1>
<h3>2018-2</h3>
</center>

## Los problemas del enfoque Naïve visto hasta ahora

Cómo vimos en el laboratorio 1, la aproximación del modelo Naïve Bayes para clasificar documentos como positivos o negativos, no fue lo suficientemente robusta para alcanzar una medidas de desempeño aceptables. La razón de esto, por lo menos para el ejercicio propuesta en el laboratorio, es que el vocabulario usado en los artículos negativos "pesó" más que el usado en los artículos positivos y el modelo presentó un sesgo hacia los documentos de esa clase, perdiendo capacidad de discriminación.

Hay que tener en cuenta que el enfoque del modelo es Naïve, como ya lo mencionamos en clases anteriores, por lo tanto, se requiere de estrategias de representación más robustas para mejorar el reconocimiento de patrones en los textos.

En esta clase vamos a hablar de nuevos enfoques de clasificación y probaremos la clasificación con nuevas bases de datos.

## Caracterización de textos más robusta

En clases anteriores hemos hablado del Part-of-Speech o etiquetado morfosintáctico y hemos mencionado algunas herramientas para realizarlo:

(En línea)
http://grupotnt.udea.edu.co/TNTagger

(La que trabajaremos a partir de ahora)

http://www.corpus.unam.mx/servicio-freeling/

(De escritorio)

TreeTagger

Freeling

NLTK (Natural Language Toolkit)

Aunque esta última opción (NLTK) es una librería potente para procesamiento de lenguaje natural, y se debe explorar (consultar), para el español, Freeling es una excelente herramienta para POS, desambiguación de sentido y análisis sintáctico; por eso la usaremos en el curso. 

El etiquetado morfosintáctico juega un papel muy importante en el proceso de caracterización o representación de un texto como vector de características para fines de reconocimiento de patrones. por ejemplo: para el análisis de sentimientos, una de las clasificaciones que nos ocupa en el curso.

## Acerca de Freeling

http://nlp.lsi.upc.edu/freeling/index.php/node/1

Es una herramienta open source para análisis de lenguaje. Permite realizar etiquetado morfosintáctico, análisis sintáctico, desambiguación de sentido, entre otros.

Para los objetivos del curso nos interesa el POS, y para ello vamos a usar una API desarrollada en la UNAM, y que nos permitirá trabajar directamente desde el notebook con Python.

http://www.corpus.unam.mx/servicio-freeling/

## ¿Cómo usar Freeling desde el notebook con la API de la UNAM?

In [7]:
#-*- coding: utf-8 -*-
import requests

#Archivo a ser enviado
files = {'file': open('prueba.txt', 'r')}

#Parámetros
params = {'outf': 'tagged', 'format': 'json'}

#Enviar petición
url = "http://www.corpus.unam.mx/servicio-freeling/analyze.php"
r = requests.post(url, files=files, params=params)
#print (r)

#Convertir de formato json
obj = r.json()

#pint(len(obj))    #obj es una lista de listas. Tiene tantos elementos (listas) como frases tenga el file
 
#Ejemplo, obtener todos los lemas
for sentence in obj:
    #print (sentence)    #sentence es una lista de diccionarios. Tiente tantos elementos (diccionarios)
                         #como palabras tiene la frase
    for word in sentence:
        #word es un diccionario con 4 claves: token, lemma, tag y prob(probabilidad de que el tag haya sido bien asignado)
        #A través de esas cuatro claves se podrá acceder a la información que requerimos para
        #Comenzar a construir el vector de características de un texto
        print (word['tag'])
        

PI0MS00
SP
DA0MP0
AQ0CP00
NCMP000
PR0CN00
PP1CS00
VAIP1S0
VMP00SM
Fc
DA0MS0
NCMS000
RG
DA00S0
RG
AQ0MS00
Fc
PP2CS00
VMIP3P0
VMN0000
RG
RG
Fp
DA0MS0
NCMS000
VSIP3S0
RG
AQ0MS00
Fc
DP3CSN
NCFS000
AQ0FS00
CC
DA0FS0
NCFS000
VSIP3S0
DA0FS0
AQ0CS00
Fp
AQ0MS00
NCMS000
VMIF1S0
RG
NCMP000
Fc
RG


De los diccionarios (word) que se muestran al ejecutar la celda anterior hay varios aspectos para analizar.

¿Qué es token?

Es cada palabra que aparece en el texto.

¿Qué es lemma?

Es la palabra raíz del token.

¿Qué es tag y cuáles hay?

https://talp-upc.gitbooks.io/freeling-4-0-user-manual/content/tagsets/tagset-es.html

¿Qué es prob?

Es la probabilidad de que el tag asignado sea correcto.

## Aspectos clásicos para la caracterización de textos para análisis de sentimientos

(Consultar el texto Sentiment Analysis - Mining Opinions, Sentiments and Emotions. Bing Liu, 2015.)

En la literatura se resportan diferentes enfoques para abordar el problema de reconocimiento de patrones en textos, conocido como análisis de sentimientos. Uno de los enfoques más usados, especialmente para textos donde se plasman revisiones de productos o servicios, es la caracterización de los textos a partir de los adjetivos. Esto es, un texto se representa con dos características:

X1: la frecuencia de aparición de adjetivos positivos (puede ser frecuencia relativa respecto al # de tokens del texto o frecuencia absoluta)

X2: la frecuencia de aparición de adjetivos negativos (puede ser frecuencia relativa respecto al # de tokens del texto o frecuencia absoluta)

Gracias al POS ya se pueden identificar de manera automática los adjetivos dentro del texto. Sin embargo, surgen una nueva pregunta.

¿El POS nos dice si el adjetivo es positivo o negativo?

La respuesta es, No. Para encontrar la "carga de sentimiento" o polaridad de los adjetivos (en general de cualquier palabra), una posible alternativa es usar lo que se conoce como bases de datos léxicas. En el curso vamos a trabajar con la base de datos conocida como MLSenticon (http://timm.ujaen.es/recursos/ml-senticon/).

Sin embargo, existen otras bases de datos léxicas (disponibles para el inglés) muy conocidas como SentiWordNet, MPQA, entre otras.

## La negación

Un aspecto fundamental cuando se trabaja en análisis de sentimientos es la negación. Como ya hemos mencionado en la introducción del curso, la identificación automática de las negaciones es fundamental para identificar la polaridad completa de una oración o de un texto, puesto que a pesar de encontrar una alta frecuencia de entradas positivas (i.e., adjetivos), si estas están negadas, entonces la polaridad del texto es negativa y no positiva.

En el curso trabajaremos con un enfoque básico reportado en la literatura, que consiste en identificar la presencia de la negación dentro de una oración y ajustar la polaridad de la misma de acuerdo a la presencia o ausencia de la negación. Esto se traduce en una nueva característica:

X3: 1 (Si hay presencia de negación en la oración) | 0 (Si no hay presencia de negación en la oración)

## Veamos algunos ejemplos de textos positivos y negativos

Los siguientes ejemplos son reales. Tomados de las calificaciones dejadas en Booking por usuarios del Hotel Dann Carlton en Medellín. (https://www.booking.com/hotel/co/dann-carlton-medellin.es.html?label=gen173nr-1DCAEoggJCAlhYSDNYBGgyiAEBmAEKwgEKd2luZG93cyAxMMgBDNgBA-gBAZICAXmoAgM;sid=05e2a52229dbab954a14b9838f09e4cb;dest_id=-592318;dest_type=city;dist=0;group_adults=2;hapos=1;hpos=1;room1=A%2CA;sb_price_type=total;srepoch=1535390967;srfid=88dea29671deb6c48a3cad6576fe653a3f08ccd9X1;srpvid=ff807afb0f3400e4;type=total;ucfs=1&#hotelTmpl)

### Revisiones de productos o servicios

### Positivos

La habitación muy cómoda y grande, la comida deliciosa, el personal muy amable. (pos)

Servicio y personal excelente. (pos)

el servico del personal es excelente, los desayunos son magnificos aunque falta un poco mas de variedad, el valor de las comidas a la carta son un poco elevados pero de gran calidad. (pos)

Es muy agradable en todos los aspectos. El servicio excelente (pos)

El servicio es muy bueno, el personal siempre esta atento y es muy amable, la comida en los restaurantes es magnifica, recomiendo el restaurante el puerto (parrilla Argentina) realmente espectacular. (pos)

La habitación es moderna y muy cómoda (pos)

Camas muy comodas, habitaciones muy limpias, personal muy amable y servicial. (pos)

Excelente ubicación, cuarto automatizado, manejas todo desde el control del Directv. La piscina es temperada lo que la hace utilizable en días no cálidos. Mis hijos disfrutaron mucho. (pos)

El control remoto del tv es inteligente y te permite controlar iluminación y blackout.
La cama lo mejor de lo mejor! (pos)

La cama es muy cómoda. Buen tipo de colchón, ya que permite descansar en forma bastante cómoda. (pos)


### Negativos

Cenamos en el Restaurant "Tony Roman" goratorio de el hotel, pero nuestra experiance <b>no</b> fue tan placentera. La comida es pasable, pero la persona que estaba cantando era extremadamente mala entonada! la musica estaba a tan alto volumen que no podiamos hablar sin gritar. Arruino nuestra cena con eso. (neg)

El turco no representa un hotel 5 estrellas. (neg)

Hotel muy grande, algún evento quizás te puede resultar molesto. Desayuno bueno <b>pero</b> podría estar mejor. (neg)

El seguro hotelero no está incluido ni descrito en el precio y aumentó un 20% la estadía con mi familia. Eso debería mejorarlo Booking. No pude usar el televisor con Internet para ver Netflix porque solo te dejan el control del Directv. (neg)

me parecieron exagerados los precios de la fuente de soda de la piscina.
no me pareció agradable los cargos extras de sorpresa q me cobraron por mi hijo de 13años. no decia nada booking. (neg) <b>Este está marcado como positivo en Booking, qué opina?</b>

Los pequeño detalles como un switch de la mesa de noche estaba dañado y otras cosas estaban algo viejas. (neg)

El servicio de atención en la piscina es un desastre! La persona que me atendió fue muy deshonesto con la cuenta, pero por la prisa al aeropuerto no pude reclamar en el lugar! (neg)

Básicamente no entiendo para que booking da la opción de escribir solicitudes especiales si el hotel no las tiene en cuenta. Escribí que quería una habitación con buena ubicación y buena vista. Y me ubicaron en la habitación más oscura del hotel en el tercer piso, la sensación de oscuridad y vista me hacía sentir que estuviera en el primer piso del hotel. Además que al lado de mi habitación (yo estaba en la 301) estaban realizando unos trabajos y escuché desde las 6;30 de la mañana martillazos. Lo cual no entiendo cómo hacen ruido en trabajos Locativos desde tan temprano. No pude descansar, me tocó levantarme temprano. (neg)

la comida no es muy buena. (neg)

El baño, específicamente el inodoro, es malo, uno tiene que que halar varias veces. La comida del restaurante argentino mala. El servicio en el comedor a diferencia de la vez pasada ha decaído. (neg)



#### Notar la importancia de las negaciones en algunos de los textos negativos.
Es por la polaridad y la negacion

#### Hay que ser concientes de las limitaciones del recurso léxico ML Senticon.

#### Notar que se pueden crear múltiples representaciones de los textos basadas en POS, no solo de un único token sino usando características poliléxicas (i.e., V+Art+Nombre, Adv+Adj, Nombre+Adj, etc.)

## Ejercicio de Clase

1. Con los 20 textos de revisiones de productos o servicios del Hotel presentados en la celda anterior, cree dos directorios, uno para los positivos y otro para los negativos, donde cada texto esté guardado en un archivo independiente con el nombre text_pos_1.txt o text_neg_1.txt según sea positivo o negativo y variando el identicador para cada archivo desde 1 hasta 10 para cada clase (pos o neg).

2. Construya un scritp en una celda de código para realizar el etiquetado morfosintáctico de todos los textos de la base de datos construída.

3. Usando el recurso léxico ML Senticon construya la representación vectorial de cada texto de la siguiente manera:

Cada texto va a estar represetado por dos características.

X1: carga positiva de los adjetivos encontrados en el texto

X2: carga negativa de los adjetivos encontrados en el texto

Tenga en cuenta que primero se deben identificar los adjetivos, luego tomar la carga que este tenga en ML Senticon, e ir sumando las cargas. Si el adjetivo no se encuentra en ML Senticon asignarle una carga de cero.

<b>Muy importante, la negación</b>. Si en una oración hay presencia de negación (no, pero, aunque), se debe invertir la polaridad de los adjetivos que se encuentren dentro de dicha oración.

No olvide tener en cuenta la etiqueta (pos o neg) de cara texto.




In [72]:
# Construya un script en una celda de código para realizar el etiquetado 
# morfosintáctico de todos los textos de la base de datos construída.
#-*- coding: utf-8 -*-
import requests

sentimientos = open('MLSenticon.txt', 'r')

MLSenticonDic = { }
for word in sentimientos:
    palabra = word.split('\t')[0]
    valor = float(word.split('\t')[1][:-1])
    MLSenticonDic[palabra] = valor


    
for i in range(10):
    vectorCaracteristicas = [0,0]
    #Archivo a ser enviado
    files = {
        'file': open('positivos/text_pos_'+ str(i+1) +'.txt', 'r')
    }

   # print(files)
    #Parámetros
    params = {'outf': 'tagged', 'format': 'json'}

    #Enviar petición
    url = "http://www.corpus.unam.mx/servicio-freeling/analyze.php"
    r = requests.post(url, files=files, params=params)
   # print (r.text)

    #Convertir de formato json
    obj = r.json()

    #pint(len(obj))    #obj es una lista de listas. Tiene tantos elementos (listas) como frases tenga el file

    #Ejemplo, obtener todos los lemas
    for sentence in obj:
        #print (sentence)    #sentence es una lista de diccionarios. Tiente tantos elementos (diccionarios)
                             #como palabras tiene la frase
        for word in sentence:
            #word es un diccionario con 4 claves: token, lemma, tag y prob(probabilidad de que el tag haya sido bien asignado)
            #A través de esas cuatro claves se podrá acceder a la información que requerimos para
            #Comenzar a construir el vector de características de un texto
            if (word['tag'][0] == 'A'):
                lema = word['lemma']
               # print(lema)
                
                try: 
                    valor = MLSenticonDic[lema]
                except:
                    valor = 0
                
                if (valor > 0):
                    vectorCaracteristicas[0] += valor
                else:
                    vectorCaracteristicas[1] += valor
                       
    print(vectorCaracteristicas)

[1.438, 0]
[0.75, 0]
[1.088, 0]
[1.5, 0]
[1.904, 0]
[0.328, 0]
[1.532, 0]
[1.125, 0]
[1.354, 0]
[0.5, 0]


In [75]:
for i in range(10):
    vectorCaracteristicas = [0,0]
    #Archivo a ser enviado
    files = {
        'file': open('negativos/text_neg_'+ str(i+1) +'.txt', 'r')
    }

   # print(files)
    #Parámetros
    params = {'outf': 'tagged', 'format': 'json'}

    #Enviar petición
    url = "http://www.corpus.unam.mx/servicio-freeling/analyze.php"
    r = requests.post(url, files=files, params=params)
   # print (r.text)

    #Convertir de formato json
    obj = r.json()

    #pint(len(obj))    #obj es una lista de listas. Tiene tantos elementos (listas) como frases tenga el file

    #Ejemplo, obtener todos los lemas
    for sentence in obj:
        #print (sentence)    #sentence es una lista de diccionarios. Tiente tantos elementos (diccionarios)
                             #como palabras tiene la frase
        for word in sentence:
            #word es un diccionario con 4 claves: token, lemma, tag y prob(probabilidad de que el tag haya sido bien asignado)
            #A través de esas cuatro claves se podrá acceder a la información que requerimos para
            #Comenzar a construir el vector de características de un texto
            if (word['tag'][0] == 'A'):
                lema = word['lemma']
               # print(lema)
                
                try: 
                    valor = MLSenticonDic[lema]
                    print(lema, valor)
                except:
                    valor = 0
                
                if (valor > 0):
                    vectorCaracteristicas[0] += valor
                else:
                    vectorCaracteristicas[1] += valor
                       
    #print(vectorCaracteristicas)

placentero 0.875
pasable 0.542
malo -0.0
grande 0.292
molesto -0.538
bueno 0.5
mejor 0.375
agradable 0.75
extra 0.25
positivo 0.594
deshonesto -0.5
especial 0.411
bueno 0.5
bueno 0.5
oscuro -0.575
bueno 0.5
malo -0.0
malo -0.0


### Recordar los modelos Logistic Regression y KNN para clasificar los textos del ejercicio y comparar con lo que haría Naïve Bayes

## Otras representaciones y otros tipos de clasificación

En el área de la recuperación de información (IR), se trabaja una clasificación no supervisada de los textos. Allí hay varias técnicas de representación y modelos que intentan resolver el problema de agrupar los textos por temas. Es importante conocer de qué se tratan estas técnicas que trabajan bajo un enfoque distinto al del POS para representar los textos.

<b>Representación Term-Document Matrix (TDM) (Matriz de términos en documentos):</b>

Es una matriz en la que cada fila representa una palabra del vocabulario y cada columna representa un documento de la colección de documentos sobre la cuál se trabaja. Esta matriz fue definida por primera en la presentación del modelo $\textit{vector space model}$ (Salton, 1971).

Veamos un ejemplo a partir de una pequeña porción de la matriz de términos en documentos para cuatro palabras en cuatro novelas de Shakespeare.

<img src="img/TDM_img_1.png">

Las marcas rojas nos dicen que cada documento se puede ver como un vector columna que está definido por la frecuencia de aparición de cada palabra de un vocabulario dentro del documento.

Para ver una representación gráfica de los documentos (novelas) vamos a tener en cuenta solo 2 dimensiones (battle y fool).

<img src="img/TDM_img_2.png">

En recuperación de información la tarea consiste en encontrar un documento $d$ dentro de una colección de documentos $D$, de tal manera que se satisfaga una consulta $q$. De esta manera, $q$ debe ser representada como un vector (documento) y posteriormente comparada con los vectores que representan los documentos $D$ para recuperar el documento $d$ (pueden ser varios documentos recuperados) más simular a $q$. Más adelante veremos cómo se puede medir la simulitud entre estos vectores que represetan los documentos.

La idea que hay detrás de la representación TDM es que dos documentos similares tiene palabras similares y por lo tanto, tendrán representaciones similares como vectores columna.

<b>Representación Term-Term Matrix (Matriz de términos en contextos):</b>

Existe una representación muy usada actualmente, conocida como embedding y que surge a partir de la matriz de terminos en contextos. En esta representación se ve cada palabra como un vector y es útil para comparar la simulitud semántica entre dos palabras y por lo tanto entre documentos que contengan esas palabras, de allí el nombre de representación en vectores semánticos.

En esta representación cada fila corresponde al vector que representa a una palabra y en las columnas se encuentran también palabras, que definen un contexto alrededor de dicha palabra en un corpus específico. Esto es, la matriz Term-Term Matrix, de dimensión |V|x|V|, tiene en cada una de sus entradas o posiciones la frecuencia de coocurrencia de la palabra de la fila con el contexto definido alrededor de la palabra de la columna. El contexto se suele definir con una ventana de entre 1 y 8 caracteres a izquierda y derecha de la palabra. Mientras más pequeña la ventana del contexto más se acerca el vector a una representación sintáctica. Por su parte, si la ventana es más amplia se acerca a una representación semántica. Veamos un ejemplo para comprender este concepto:

<img src="img/TTM_img_1.png">

Contexto definido como una ventana de 7 tokens a izquierda y derecha.

<img src="img/TTM_img_2.png">

Porción de Term-Term Matrix. Notar que esta matriz es una matriz sparse. 

Veamos la representación visual de los vectores semánticos de las palabras digital & information.

<img src="img/TTM_img_3.png">

Esta gráfica se interpreta así:

la palabra digital coocurre una vez con la palabra result en un contexto definido a partir de una ventana de 7 caracteres a izquierda y derecha. De igual manera coocurre una vez con la palabra data.

Por su parte, la palabra information, coocurre 4 veces con la palabra result y 6 veces con la palabra data dentro de los contextos definidos bajo la misma ventaja.

Notar que el vector que representa una palabra es del tamaño del vocabulario (|V|), regularmente entre 10.000 y 50.000 entradas, esto si se usan las palabras más frecuentes en un corpus de entrenamiento. Por esta razón es que la matriz de términos en contextos es sparse y se requiere de algoritmos eficientes para trabajar con ella.

Si bien la matriz de términos en contextos nos da una medida de asociación entre dos palabras, está basada únicamente en una frecuencia de coocurrencia y esto en muchos contextos no es suficiente. Por tal razón, existen otras medidas más robustas como Positive Pointwise Mutual Information (PPMI) o TF-IDF.

<b>Term Frequency - Inverse Term Frequency Scheme (TF-IDF)</b>

Esta medida está definida como el producto de dos factores:

TF: la frecuencia de una palabra en un documento. (se suele usar el logarítmo de esa frecuencia)
IDF: el cociente entre el # total de documentos ($N$) y el número de documentos en los cuáles aparece la palabra ($df_i$).

$$idf_i = log(\frac{N}{df_i})$$

Al combinar $TF$ con $IDF$ surge el esquema de pesado $tf-idf$ para una palabra $w_i$ en un documento $j$.

$$w_{ij} = tf_{ij}idf_i  $$



## Para medir similaridad: el coseno

Debido a que las palabras se pueden representar como vectores semánticos, se puede medir similaridad entre esas palabras si se mide la simularidad entre los vectores que las representan. La medida más usada en el procesamiento de lenguaje natural para medir esa similaridad es el coseno. A partir de la definición del producto interno visto en álgebra lineal:

<img src="img/cos.png">

Por lo tanto,

<img src="img/cos_2.png">

Considere el siguiente ejercicio:

A partir de la medida de similitud del coseno determine cual de las dos palabras, apricot o digital, está más cerca semánticamente de la palabra information. Tenga en cuenta la representación de la siguiente tabla.

<img src="img/ejemplo_cos.png"><br><!-- <img src="ejemplo_cos_result.png"> <br><img src="ejemplo_cos_result_graph.png">-->




### Referencias

Jurafsky, D., & Martin, J. H. (2014). Speech and language processing (Vol. 3). London: Pearson.